# White Island Soil Gas Variation Plot

## Purpose: ##
Show all observations as a pseudo-xy plot. The x-axis represents the site (siteID), the y-axis represents the flux. Each plotted line represents the values from a single survey. The values from the latest survey are shown in a distinguishing colour.

## Author: ##
Steven Sherburn

## Date: ##
July 2018

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
import requests

%matplotlib inline

In [ ]:
#full width notebook display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

**Data search parameters**

In [ ]:
typeID = 'CO2-flux-e'
methodID = 'chmb'
latmin = '-37.528'
latmax = '-37.518'
lonmin = '177.184'
lonmax = '177.196'

within = 'POLYGON(('+lonmin+'+'+latmax+','+lonmin+'+'+latmin+','+lonmax+'+'+latmin+','+lonmax+'+'+latmax+','+lonmin+'+'+latmax+'))'

startdate = '2017-01-26'

**Query**

In [ ]:
url = 'http://fits.geonet.org.nz/site'
payload = {'typeID': typeID, 'methodID': methodID, 'within': within}

**Get sites from site query**

In [ ]:
r = requests.get(url,params=payload)
jdata = r.json()
features = jdata['features']
dfsite = pd.DataFrame() #empty dataframe
for i, val in enumerate(features):
  geometry = val['geometry']
  lon = geometry['coordinates'][0]
  lat = geometry['coordinates'][1]
  properties = val['properties']
  siteID = properties['siteID']
  height = properties['height']
  name = properties['name']
  #append these to df
  dfsite = dfsite.append({'siteID': siteID, 'lon': lon, 'lat': lat, 'height': height, 'name': name}, ignore_index=True)

In [ ]:
dfsite.tail()

**For each siteID, get time-series data and accumulate**

In [ ]:
names = ['dt', 'obs', 'err']
dfts = pd.DataFrame()   #empty dataframe

In [ ]:
for site in dfsite['siteID']:
  #site = site.encode('ascii','ignore')
  #print site, network
  url= 'https://fits.geonet.org.nz/observation?typeID='+typeID+'&siteID='+site+'&methodID='+methodID
  df = pd.read_csv(url, names=names, skiprows=1, parse_dates={"Datetime" : ['dt']})
  df['siteID'] = site
  dfts = dfts.append(df, ignore_index=True)

**Merge the two dataframes on siteID, keep only those after specified time**

In [ ]:
dfall = pd.merge(dfsite, dfts, on='siteID')
dfall = dfall[dfall['Datetime']>startdate]

dfall.drop(['height', 'lat', 'lon', 'name', 'err'], axis='columns', inplace=True)

**Get final observations**

In [ ]:
dates = dfall['Datetime'].unique()
datelist = dates.astype('datetime64[D]').astype('str').tolist()
datelist.sort()

#date of last values
last = datelist[-1]

dflast = dfall[dfall['Datetime']==last]

dflast.tail()

In [ ]:
#index from 1 to n
# dflast.index = range(1,len(dflast)+1)
dflast.index = range(len(dflast))

In [ ]:
dfall.groupby('siteID')['obs'].count().head()

In [ ]:
fig, ax = plt.subplots(figsize=(25,5))
for date in datelist:
    d = dfall[dfall['Datetime']==date]
    ax.plot(d['siteID'], d['obs'], color='blue', alpha=0.5, label='')
    
#overplot last data
dflast['obs'].plot(ax=ax, color='red', rot=90, label='Latest')

ax.set_title('White Island Soil Gas Flux, All Data By SiteID', fontsize=16)
ax.set_ylabel('Flux (g/m^2/d)')
plt.legend()

plt.savefig('soilgas_all_by_siteid.png')